# Reproduction of Tutorial Example using Lifelines Library

## General Steps to Conduct Survival Analysis
1. Load and preprocess the data
2. Estimate the survival function using Kaplan-Meier estimator
3. Plot the survival curve
4. Perform statistical comparisons using log-rank test (if applicable) 
5. Fit various parametric survival models (e.g., Weibull, Exponential, Log-Normal, Log-Logistic, etc.)
6. Compare the models to select the best model
7. Fit Cox's proportional hazard model
8.  Check the proportional hazard assumption
9.  Predict survival function for new data
10. Evaluate the model predictive performance
11. Interpret the results
12. Conclusion

In [ ]:
# Install lifelines
# !pip install lifelines

In [1]:
# Import libraries
import lifelines
print(lifelines.__version__)  # Should print the installed version


0.30.0


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys # for sys.path.append to work

## Load and Preprocess the Data